In [2]:
import torch
from dlc_practical_prologue import generate_pair_sets
import torch.nn as nn

## 1. Generate raw data, process it

In [3]:
train_input_0, train_target_0, train_classes_0, test_input_0, test_target_0, test_classes_0 = generate_pair_sets(1000)

In [4]:
def normalize_data(tensor):
    mu, std = tensor.mean(), tensor.std()
    tmp = tensor.sub(mu).div(std)
    return tmp

In [5]:
def convert_to_one_hot_labels(target):
    tmp = target.new_zeros(target.size(0), target.max() + 1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

_We normalized the data so it has mean 0 and std 1._

In [6]:
train_input = normalize_data(train_input_0)
# train_target = convert_to_one_hot_labels(train_target_0)
train_target = train_target_0

test_input = normalize_data(test_input_0)
# test_target = convert_to_one_hot_labels(test_target_0)
test_target = test_target_0

In [7]:
print("Training set mean = {a}".format(a = train_input.mean().item()))
print("Training set std = {s}\n".format(s = train_input.std().item()))
print("Test set mean = {a}".format(a = test_input.mean().item()))
print("Test set std = {s}".format(s = test_input.std().item()))

Training set mean = -1.1463554550061872e-08
Training set std = 1.0

Test set mean = 2.992591134898248e-07
Test set std = 0.9999999403953552


## 2. Creating & Training models

In [8]:
import models
import torch.nn.functional as F

In [72]:
basic_model = models.BaselineNetwork()

In [73]:
def train_model(model, train_input, train_target, train_classes, nb_epoch, batch_size, optimizer_params):
    nb_epoch, batch_size = nb_epoch, batch_size
    lr, momentum = optimizer_params['lr'], optimizer_params['momentum']
    
    optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum = momentum)
    criterion = nn.CrossEntropyLoss()
    
    for e in range(nb_epoch):
        for inputs, targets in zip(train_input.split(batch_size),
                                  train_target.split(batch_size)):
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [74]:
train_model(basic_model, train_input, train_target, train_classes_0, 50, 32, {'lr': 0.01, 'momentum':0.9})

In [75]:
BATCH_SIZE = 10
LOG_INTERVAL = 20

In [76]:
def test(test_input, test_target, test_classes, model, criterion):
    
    with torch.no_grad():
        nb_data_errors = 0
        loss_sum = 0
        for b in range(0, test_input.size(0), BATCH_SIZE):
            output = model(test_input.narrow(0, b, BATCH_SIZE))
            loss = criterion(output, test_target.narrow(0, b, BATCH_SIZE))
            loss_sum += loss
            _, predicted_classes = torch.max(output, 1)

            for k in range(BATCH_SIZE):
                #print(repr(test_target[b + k]) + " - " + repr(predicted_classes[k]))
                if test_target[b + k] != predicted_classes[k]:
                    nb_data_errors = nb_data_errors + 1

            if b % LOG_INTERVAL == 0:
                pass
                #print("Accuracy: " + repr())
                #print_test(batch_idx, len(val_loader), batch_time, losses, top1, persistent=False, color=color, title=title)
    
#         print("Nb errors: " + repr(nb_data_errors))
        accuracy = 1 - (nb_data_errors / test_input.size(0))
#         print("Accuracy: " + repr(accuracy * 100) + "%" + " - Loss: " + repr(loss_sum.item()))
        
        return (accuracy * 100)

In [77]:
test(test_input, test_target, test_classes_0, basic_model, nn.CrossEntropyLoss())

80.4

In [78]:
learning_rates = [0.001, 0.01] #, 0.1]
momentums = [0.9] #[0.5, 0.7, 0.9]
nb_epochs = [20, 50] #, 100]
batch_sizes = [16, 32] #, 64]

best_accuracy = 0
best_params = {}

for lr in learning_rates:
    for momentum in momentums:
        for nb_epoch in nb_epochs:
            for batch_size in batch_sizes:
                optimizer_params = {'lr':lr, 'momentum':momentum}
                
                model = models.BaselineNetwork()
                
                train_model(model, train_input, train_target, train_classes_0, nb_epoch, batch_size, optimizer_params)
                
                accuracy = test(test_input, test_target, test_classes_0, model, nn.CrossEntropyLoss())
                
                print(lr)
                print(momentum)
                print(nb_epoch)
                print(batch_size)
                print("\n")
                
                
                
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['lr'] = lr
                    best_params['momentum'] = momentum
                    best_params['nb_epoch'] = nb_epoch
                    best_params['batch_size'] = batch_size
                    
print("Best accuracy obtained = {a}\n".format(a = best_accuracy))
print("with the following hyperparameters:\n")
print(best_params)

0.001
0.9
20
16


0.001
0.9
20
32


0.001
0.9
50
16


0.001
0.9
50
32


0.01
0.9
20
16


0.01
0.9
20
32


0.01
0.9
50
16


0.01
0.9
50
32


Best accuracy obtained = 80.89999999999999

with the following hyperparameters:

{'lr': 0.001, 'momentum': 0.9, 'nb_epoch': 50, 'batch_size': 16}


## 3. Testing model

_In order to test the model we will generate new data (training and test set), retrained the model on the new data en evaluate it on the new test set. We will do this process more than 10 times and estimates the mean accuracy as well as its standard deviation._

In [29]:
nb_rounds = 10
test_model = models.BaselineNetwork

In [42]:
def evaluate_model(model, nb_rounds, criterion):
    
    accuracies = []
    
    for round in range(nb_rounds):
        
        # initialize new model
        model_evaluated = model()
        # generate new data
        train_input, train_target, train_classes, test_input, test_target, test_classes = generate_pair_sets(1000)
        train_input = normalize_data(train_input)
        test_input = normalize_data(test_input)
        
        train_model(model_evaluated, train_input, train_target, train_classes)
        
        accuracy = test(test_input, test_target, test_classes, model_evaluated, criterion)
        
        print("Round {i}: accuracy = {a}".format(i = (round + 1), a = accuracy))
        
        accuracies.append(accuracy)
        
    return torch.FloatTensor(accuracies)

In [43]:
accuracies = evaluate_model(test_model, nb_rounds, nn.CrossEntropyLoss())

Round 1: accuracy = 79.60000000000001
Round 2: accuracy = 80.0
Round 3: accuracy = 77.7
Round 4: accuracy = 79.60000000000001
Round 5: accuracy = 79.5
Round 6: accuracy = 74.9
Round 7: accuracy = 80.89999999999999
Round 8: accuracy = 79.5
Round 9: accuracy = 80.1
Round 10: accuracy = 80.0


In [45]:
print("The mean accuracy is: {a}".format(a = accuracies.mean()))
print("The accuracy std is: {s}".format(s = accuracies.std()))

The mean accuracy is: 79.18000030517578
The accuracy std is: 1.7067185640335083
